# 機能/目的
## - 学習済みインテントを利用してチャットボットを自動構成する

## 0 前提作業


### 1) Watson SDKの導入

In [ ]:
!pip install --upgrade "watson-developer-cloud>=1.3.0" 

### 2) クリデンシャル情報のセット

In [ ]:
# -*- coding: utf-8 -*-
import json
import re
import watson_developer_cloud

assistant = watson_developer_cloud.AssistantV1(
    username='',
    password='',
    version= '2018-02-16'
)

### 3) workspace作成

In [ ]:
workspace = assistant.create_workspace(
    language = "ja",
    name= "ダイアログ自動生成デモ",
    description ='2018/4/24版 学習済みインテント、システムエンティティ、フォルダ、脱線',
    entities = [
        { 'entity' : "sys-date" },
        { 'entity' : "sys-time" },
        { 'entity' : "sys-currency" },
        { 'entity' : "sys-percentage" },
        { 'entity' : "sys-number" }
    ]
)

workspaceid = workspace["workspace_id"]

## 1 workspaceを開いて学習済みインテントをインポートする

- 学習済みインテント
    - 一般とe-コマースのインテントを取り込む

## 2 初期ノード作成 (welcome, anything_else)

In [ ]:
mynode = assistant.create_dialog_node(
    workspace_id = workspaceid,
    title = "ようこそ",
    conditions = "welcome",
    output = {'text':"こんにちは、ワトソンです"},
    dialog_node = "start"
)
mynode = assistant.create_dialog_node(
    workspace_id = workspaceid,
    title = "その他",
    conditions = "anything_else",
    output = {'text':"解釈できません"},
    dialog_node = "end",
    previous_sibling = "start"
)

## 3 フォルダ作成 for Pre-Trained Intents

In [ ]:
response = assistant.list_intents(
    workspace_id = workspaceid
)

#フォルダ名の生成
myintent = ""
myintents = []
for index, intent in enumerate(response["intents"]):
    index = intent["intent"].find('_')
    if (intent["intent"][:index] != myintent):
        myintent = intent["intent"][:index]
        myintents.append(myintent)

#フォルダの作成
for fname in myintents:
    assistant.create_dialog_node(
        workspace_id = workspaceid,
        title = fname,
        conditions = "true",
        dialog_node = fname,
        node_type = "folder",
        digress_in = "returns",
        previous_sibling = "start"
    )
    print ("Creating folder ... ", fname)

## 4 学習済みインテント用ダイアログノードの作成

In [ ]:
for fname in myintents:
    for index, intent in enumerate(response["intents"]):
        nodename = intent["intent"]
        if nodename.find(fname) > -1:
            myparent = fname
            assistant.create_dialog_node(
                workspace_id = workspaceid,
                dialog_node = nodename,
                conditions = '#' + nodename,
                output = { 'text': "まだ学習していません"},
                title = intent["description"],
                parent = myparent
            )
        print ("Creating node ... ", nodename)

## 5 ジョーク対応

In [ ]:
response = assistant.update_dialog_node(
    workspace_id = workspaceid,
    dialog_node = 'General_Greetings',
    new_output = {
        'text': '僕はWatson Studioから自動生成されたボットだよ'
    }
)

response = assistant.update_dialog_node(
    workspace_id = workspaceid,
    dialog_node = 'General_Jokes',
    new_output = {
        'text': 'まだジョークは学習していないんですよ'
    }
)

## 6 購入対応

In [ ]:
response = assistant.update_dialog_node(
    workspace_id = workspaceid,
    dialog_node = 'eCommerce_Create_Product_Order',
    new_output = {
        'text': '在庫があるのでお持ち帰りできますよ'
    }
)

## 7 返品対応

In [ ]:
response = assistant.update_dialog_node(
    workspace_id = workspaceid,
    dialog_node = 'eCommerce_Cancel_Product_Order',
    new_output = {
        'text': 'いつ購入しましたか'
    }
)
response = assistant.create_dialog_node(
    workspace_id = workspaceid,
    dialog_node = 'eCommerce_Check_Create_Product_Order',
    parent = 'eCommerce_Cancel_Product_Order',
    conditions = '@sys-date',
    output = {},
    #digress_out = 'allow_all',
    title = '購入日の確認'
)
response = assistant.create_dialog_node(
    workspace_id = workspaceid,
    dialog_node = 'eCommerce_Check_Create_Product_Order_1',
    parent = 'eCommerce_Check_Create_Product_Order',
    node_type = 'response_condition',
    conditions = "@sys-date.after('2018-04-07')",
    output = {
        'text': '未使用であれば返却できます'
    }
)
response = assistant.create_dialog_node(
    workspace_id = workspaceid,
    dialog_node = 'eCommerce_Check_Create_Product_Order_2',
    parent = 'eCommerce_Check_Create_Product_Order',
    node_type = 'response_condition',
    output = {
        'text': '残念ながら返却できません'
    },
    previous_sibling = 'eCommerce_Check_Create_Product_Order_1'
)

## [注意] 全ノードの削除
- 必要な場合にのみ実行

In [ ]:
response = assistant.list_dialog_nodes(
    workspace_id = workspaceid
)

for node in response["dialog_nodes"]:
    if (node["type"] == "folder" or node["dialog_node"] == "end" or node["dialog_node"] == "start"):
        nodename = node["dialog_node"]
        response = assistant.delete_dialog_node(
            workspace_id = workspaceid,
            dialog_node = nodename
        )

## [注意] ワークスペースの削除
- 必要な場合にのみ実行

In [ ]:
response = assistant.delete_workspace(
    workspace_id = workspaceid
)

Copyright © 2018 IBM. This notebook and its source code are released under the terms of the MIT License.